In [1]:
import netCDF4 as nc
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import xarray as xr
from IO_functions import *
from parameters import *
from statsFunc import crossValidateKfold


Load data

In [2]:
tas1=month_load("C:/Users/predr/Desktop/",mohc_dates,most_dates,years,'tas','ICHEC-EC-EARTH','rcp45','r12i1p1','SMHI-RCA4','v1','2008','12')
tas2=month_load("C:/Users/predr/Desktop/",mohc_dates,most_dates,years,'tas','ICHEC-EC-EARTH','rcp85','r12i1p1','SMHI-RCA4','v1','2008','12')
hurs=month_load("C:/Users/predr/Desktop/",mohc_dates,most_dates,years,'hurs','CNRM-CERFACS-CNRM-CM5','rcp45','r1i1p1','SMHI-RCA4','v1','2008','12')

In [ ]:
x=[]
x=xr.concat([tas1, tas2], dim="time")
#x['tas']

In [ ]:
coef=np.zeros((90,134,2))
adjr2=np.zeros((90,134))
rmse=np.zeros((90,134))

for lat in range(90):
    for lon in range(134):
        tas_cell_month=tas1['tas'][:,lat,lon]
        hurs_cell_month=hurs['hurs'][:,lat,lon]
        print(np.concatenate([tas_cell_month,tas_cell_month]))

In [47]:
tas_all_month

<xarray.Dataset>
Dimensions:    (bnds: 2, lat: 90, lon: 134, time: 124)
Coordinates:
  * time       (time) datetime64[ns] 2009-12-01T12:00:00 ... 2009-12-31T12:00:00
  * lon        (lon) float64 -22.0 -21.5 -21.0 -20.5 ... 43.0 43.5 44.0 44.5
  * lat        (lat) float64 27.0 27.5 28.0 28.5 29.0 ... 70.0 70.5 71.0 71.5
    height     float64 2.0
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) datetime64[ns] 2009-12-01 2009-12-02 ... 2010-01-01
    tas        (time, lat, lon) float32 292.1 292.2 292.2 ... 274.8 274.7 274.6
Attributes:
    CDI:                            Climate Data Interface version 1.9.6 (htt...
    history:                        Wed Dec 02 05:02:20 2020: cdo remapnn,gri...
    institution:                    Swedish Meteorological and Hydrological I...
    Conventions:                    CF-1.4
    contact:                        rossby.cordex@smhi.se
    creation_date:                  2013-07-03-T23:48:30Z
    experiment:                     RCP4.5
    experiment_id:                  rcp45
    driving_experiment:             CNRM-CERFACS-CNRM-CM5, rcp45, r1i1p1
    driving_model_id:               CNRM-CERFACS-CNRM-CM5
    driving_model_ensemble_member:  r1i1p1
    driving_experiment_name:        rcp45
    frequency:                      day
    institute_id:                   SMHI
    model_id:                       SMHI-RCA4
    rcm_version_id:                 v1
    project_id:                     CORDEX
    CORDEX_domain:                  EUR-11
    product:                        output
    references:                     http://www.smhi.se/en/Research/Research-d...
    tracking_id:                    0a9ffd2d-8fbb-4573-ba8b-8344ebfee6f9
    rossby_comment:                 201144: CORDEX Europe 0.11 deg | RCA4 v1 ...
    rossby_run_id:                  201144
    rossby_grib_path:               /nobackup/rossby16/rossby/joint_exp/corde...
    CDO:                            Climate Data Operators version 1.9.6 (htt...

In [59]:
CORDEX_path="C:/Users/predr/Desktop/"
# Parameters
folds= 10
coef=np.zeros((60,90,134,2))
adjr2=np.zeros((60,90,134))
rmse=np.zeros((60,90,134))
month_count=0 #start counting months

for year in range(2006,2010):
    for month in range(1,13):
        tas_all_month=[]
        hurs_all_month=[]
        for gcm in gcms:
            for rcp in rcps:
                for rp in rps:
                    for rcm in rcms:
                        for version in versions:
                            # Load data
                            try:
                                tas=month_load(CORDEX_path,mohc_dates,most_dates,years,'tas',gcm,rcp,rp,rcm,version,str(year),str(month))
                                hurs=month_load(CORDEX_path,mohc_dates,most_dates,years,'hurs',gcm,rcp,rp,rcm,version,str(year),str(month))
                                # Concatenate
                                if any(tas_all_month):
                                    tas_all_month=xr.concat([tas_all_month, tas], dim="time")
                                else:
                                    tas_all_month=tas
                                if any(hurs_all_month):
                                    hurs_all_month=xr.concat([hurs_all_month, hurs], dim="time")
                                else:
                                    hurs_all_month=hurs
                            except Exception as e: 
                                print(e)
        # Do stuff with monthly data here
        # Regression
        # Might have to do per cell
        #print(tas_all_month.sizes)
        for lat in range(1):
            for lon in range(5):
                tas_cell_month=tas_all_month['tas'][:,lat,lon]
                hurs_cell_month=hurs_all_month['hurs'][:,lat,lon]
                try:
                    coef[month,lat,lon,:],adjr2[month,lat,lon],rmse[month,lat,lon]=crossValidateKfold(tas_cell_month,hurs_cell_month,folds)  
                    print(str(month) + '/' + str(year) + ' finished')     
                except Exception as e: 
                    print(e)
                    pass
        month_count=month_count+1 # increment month

                #print(str(month) + '/' + str(year) + ' finished')


list' object has no attribute 'dims'
'list' object has no attribute 'dims'
'list' object has no attribute 'dims'
'list' object has no attribute 'dims'
'list' object has no attribute 'dims'
'list' object has no attribute 'dims'
'list' object has no attribute 'dims'
'list' object has no attribute 'dims'
'list' object has no attribute 'dims'
'list' object has no attribute 'dims'
'list' object has no attribute 'dims'
4/2009 finished
4/2009 finished
4/2009 finished
4/2009 finished
4/2009 finished
'list' object has no attribute 'dims'
'list' object has no attribute 'dims'
'list' object has no attribute 'dims'
'list' object has no attribute 'dims'
'list' object has no attribute 'dims'
'list' object has no attribute 'dims'
'list' object has no attribute 'dims'
'list' object has no attribute 'dims'
'list' object has no attribute 'dims'
'list' object has no attribute 'dims'
'list' object has no attribute 'dims'
'list' object has no attribute 'dims'
'list' object has no attribute 'dims'
'list' ob

In [71]:
monthly_mask = xr.open_dataset("C:/Users/predr/Desktop/CNRM-CERFACS-CNRM-CM5/merged_monthly_time.nc")
monthly_mask['time'][2:1097]

<xarray.DataArray 'time' (time: 1095)>
array(['2006-03-16T12:00:00.000000000', '2006-04-16T00:00:00.000000000',
       '2006-05-16T12:00:00.000000000', ..., '2097-03-16T12:00:00.000000000',
       '2097-04-16T00:00:00.000000000', '2097-05-16T12:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2006-03-16T12:00:00 ... 2097-05-16T12:00:00
    height   float64 ...
Attributes:
    standard_name:  time
    long_name:      time
    bounds:         time_bnds
    axis:           T

In [55]:
monthly_mask['time'].to_netcdf("C:/Users/predr/Desktop/CNRM-CERFACS-CNRM-CM5/merged_monthly_time.nc")

In [57]:
foo = xr.DataArray(adjr2, coords=[monthly_mask['time'], hurs_all_month['lat'],hurs_all_month['lon']], dims=["time", "lat","lon"])

ValueError: conflicting sizes for dimension 'time': length 60 on the data but length 1140 on coordinate 'time'

In [48]:
ds = xr.Dataset(
    data_vars=dict(
        adjr2=(["time", "lat", "lon"], adjr2),
        rmse=(["time", "lat", "lon"], rmse),
    ),
    coords=dict(
        lon=(['lon'], hurs_all_month['lon']),
        lat=(["lat"], hurs_all_month['lat']),
        time=monthly_mask['time'],
    ),
    attrs=dict(description="Weather related data."))

In [50]:
ds.to_netcdf("C:/Users/predr/Desktop/CNRM-CERFACS-CNRM-CM5/test_output.nc")

In [66]:
range(10,11)[0]

10